# Fixations

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))
from config import *
from features.feature_utils import *

In [7]:
experiment = "FIXATIONS"
participant_ids = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_samples.pq", 
        columns=["participant_id"]
    )

participant_ids = participant_ids["participant_id"].unique()

In [8]:
def get_samples_df(participant_id, experiment):
    df_samples = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_samples.pq",
        filters=[('participant_id', '=', participant_id)]
        )
    return df_samples
    
def get_events_df(participant_id, experiment):
    df_events = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_events.pq",
        filters=[('participant_id', '=', participant_id)]
        )
    
    return df_events

In [9]:
participant_id = participant_ids[0]

df_samples = get_samples_df(participant_id, experiment)
df_events = get_events_df(participant_id, experiment)
df_combined = combine_samples_events(df_samples, df_events, experiment)

In [10]:
get_distance_to_stimulus_features(df_combined)

,experiment,participant_id,distance_to_fixpoint_mean,distance_to_fixpoint_min,distance_to_fixpoint_max,distance_to_fixpoint_median,distance_to_fixpoint_std
0,FIXATIONS,106,56.324557,6.363988,132.128261,45.086285,31.72709
